In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nexus-by-djs-nsdc-ultraceuticals/sample_submission.csv
/kaggle/input/nexus-by-djs-nsdc-ultraceuticals/train.csv
/kaggle/input/nexus-by-djs-nsdc-ultraceuticals/test.csv


# Stacking

In [3]:
import pandas as pd
data = pd.read_csv('/kaggle/input/nexus-by-djs-nsdc-ultraceuticals/train.csv')
data.head()

,ID,TargetID,DRUGID,DRUGTYPE,Drug_high_status,DRUGNAME,PUBCHCID,Disease_of_highest_status,Drug_Status,UNIPROID,TARGNAME,GENENAME,SYNONYMS,FUNCTION,BIOCLASS,SEQUENCE,Disease,Accession Number,Target_Status
0,140736,T51115,D0L4YD,Small molecular drug,Approved,Solifenacin,154059,Overactive bladder,Approved,CAC1C_HUMAN,Voltage-gated calcium channel alpha Cav1.2 (CA...,CACNA1C,Voltage-gated calcium channel subunit alpha Ca...,Mediates influx of calcium ions into the cytop...,Voltage-gated ion channel,MVNENTRMYIPEENHQGSNYGSPRPAHANMNANAAAGLAPEHIPTP...,Genetic cardiac arrhythmia,Q13936,Terminated
1,133048,T60529,D03NMM,Small molecular drug,Investigative,AM-643,46843035,Dermatological disease,Investigative,PGH1_HUMAN,Prostaglandin G/H synthase 1 (COX-1),PTGS1,Prostaglandin-endoperoxide synthase 1; Prostag...,Converts arachidonate to prostaglandin H2 (PGH...,Paired donor oxygen oxidoreductase,MSRSLLLWFLLFLLLLPPLPVLLADPGAPTPVNPCCYYPCQHQGIC...,Rheumatoid arthritis,P23219,Approved
2,60493,T80975,D0T2ER,Small molecular drug,Phase 1,TAK-593,24767976,Solid tumour/cancer,Phase 1,VGFR2_HUMAN,Vascular endothelial growth factor receptor 2 ...,KDR,VEGFR2; VEGFR-2; VEGF-2 receptor; Protein-tyro...,Plays an essential role in the regulation of a...,Kinase,MQSKVLLAVALWLCVETRAASVGLPSVSLDLPRLSIQKDILTIKAN...,Renal cell carcinoma,P35968,Approved
3,169176,T92072,D07ESH,Small molecular drug,Discontinued in Phase 3,PF-1913539,176408,Alzheimer disease,Discontinued in Phase 3,AA1R_HUMAN,Adenosine A1 receptor (ADORA1),ADORA1,Adenosine receptor A1; A(1) adenosine receptor,The activity of this receptor is mediated by G...,GPCR rhodopsin,MPPSISAFQAAYIGIEVLIALVSVPGNVLVIWAVKVNQALRDATFC...,Hyper-lipoproteinaemia,P30542,Phase 2
4,120183,T30082,D0Q0RC,Small molecular drug,Approved,Ethopropazine,3290,Parkinson disease,Approved,ACES_HUMAN,Acetylcholinesterase (AChE),ACHE,YT; N-ACHE; ARACHE,Role in neuronal apoptosis. Terminates signal ...,Carboxylic ester hydrolase,MRPPQCLLHTPSLASPLLLLLLWLLGGGVGAEGREDAELLVTVRGG...,Oesophageal/gastroduodenal disorder,P22303,Approved


In [4]:
le_cols = []
for col in data.columns:
    if col=='ID' or col=='Target_Status':
        pass
    else : 
        le_cols.append(col)
le_cols

['TargetID',
 'DRUGID',
 'DRUGTYPE',
 'Drug_high_status',
 'DRUGNAME',
 'PUBCHCID',
 'Disease_of_highest_status',
 'Drug_Status',
 'UNIPROID',
 'TARGNAME',
 'GENENAME',
 'SYNONYMS',
 'FUNCTION',
 'BIOCLASS',
 'SEQUENCE',
 'Disease',
 'Accession Number']

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.drop('Target_Status', axis=1), data['Target_Status'],
                                                    stratify=data['Target_Status'], random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(100864, 18) (33622, 18) (100864,) (33622,)


In [5]:
from sklearn.preprocessing import LabelEncoder

for col in le_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = X_test[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1 if pd.notna(x) else -1)
    print("Done for", col)

Done for TargetID
Done for DRUGID
Done for DRUGTYPE
Done for Drug_high_status
Done for DRUGNAME
Done for PUBCHCID
Done for Disease_of_highest_status
Done for Drug_Status
Done for UNIPROID
Done for TARGNAME
Done for GENENAME
Done for SYNONYMS
Done for FUNCTION
Done for BIOCLASS
Done for SEQUENCE
Done for Disease
Done for Accession Number


In [6]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

print(le.classes_)

['Application submitted' 'Approved' 'Approved (orphan drug)'
 'BLA submitted' 'Clinical trial' 'Discontinued in Phase 1'
 'Discontinued in Phase 1/2' 'Discontinued in Phase 2'
 'Discontinued in Phase 2/3' 'Discontinued in Phase 2b'
 'Discontinued in Phase 3' 'Discontinued in Phase 4'
 'Discontinued in Preregistration' 'IND submitted' 'Investigative'
 'Patented' 'Phase 0' 'Phase 1' 'Phase 1/2' 'Phase 1/2a' 'Phase 1b'
 'Phase 1b/2a' 'Phase 2' 'Phase 2/3' 'Phase 2a' 'Phase 2b' 'Phase 3'
 'Phase 4' 'Preclinical' 'Preregistration' 'Registered' 'Terminated'
 'Withdrawn from market']


In [7]:
keep_cols = ['ID', 'TargetID', 'SYNONYMS',
             'UNIPROID', 'TARGNAME', 'GENENAME',  
             'FUNCTION', 'BIOCLASS',
             'Disease', 'Accession Number',]
len(keep_cols)

10

In [8]:
X_train_dropped = X_train[keep_cols]
X_test_dropped = X_test[keep_cols]

In [9]:
X_train_dropped.head()

,ID,TargetID,SYNONYMS,UNIPROID,TARGNAME,GENENAME,FUNCTION,BIOCLASS,Disease,Accession Number
88082,131977,397,293,511,499,549,143,72,271,253
36973,189122,492,447,252,228,250,661,58,260,362
76161,112276,177,248,396,389,1,176,0,515,128
5088,78657,424,565,681,660,249,364,58,155,220
76923,39756,13,59,202,186,200,171,45,384,355


In [10]:
y_train[:5]

array([ 1,  1, 17,  7, 14])

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


# Define base models
param1 = {'max_depth': 19, 'learning_rate': 0.018605498363005686, 'n_estimators': 441, 'subsample': 0.9987709321276286,
          'colsample_bytree': 0.7824142463846468, 'gamma': 0.8348828876624594, 'min_child_weight': 3, 'verbosity':2}

param2 =  {'max_depth': 19, 'learning_rate': 0.023064636414923647, 'n_estimators': 447, 'subsample': 0.9992514964718675, 
           'colsample_bytree': 0.7931212254410178, 'gamma': 0.8275527074142363, 'min_child_weight': 3, 'verbosity':2}

param3 = {'iterations': 374, 'depth': 10, 'learning_rate': 0.47497302209777004, 'l2_leaf_reg': 2.8493972804552454, 'verbose':2} 

param4 = {'iterations': 901, 'depth': 9, 'learning_rate': 0.15393581334059922, 'l2_leaf_reg': 0.6611236899302668, 'verbose':2}

param5 = {'iterations': 951, 'depth': 9, 'learning_rate': 0.18509019397073728, 'l2_leaf_reg': 0.12336456448381483, 'verbose':2}

m1 = XGBClassifier(**param1)
m2 = XGBClassifier(**param2)
m3 = CatBoostClassifier(**param3)
m4 = CatBoostClassifier(**param4)
m5 = CatBoostClassifier(**param5)


# Define the meta-learner (can be any classifier)
meta_learner = XGBClassifier()

# Create the stacking ensemble
stacking_clf = StackingClassifier(estimators=[
    ('xgb1', m1),
    ('xgb2', m2), 
    ('catboost1', m3),
    ('catboost2', m4),
    ('catboost3', m5),
], final_estimator=meta_learner, cv=2)

# Train the stacking model
# stacking_clf.fit(X_train_dropped, y_train)

# Make predictions with the stacked model
# final_predictions = stacking_clf.predict(X_test_dropped)

In [24]:
# from sklearn.metrics import accuracy_score, f1_score

# print("Accuracy of Stacked Model : ", accuracy_score(y_test, final_predictions))
# print("F1 score of Stacked Model : ", f1_score(y_test, final_predictions))

Accuracy of Stacked Model :  0.9925941347926952


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [8]:
print("Done")

Done


# Test set predictions

In [9]:
test = pd.read_csv('/kaggle/input/nexus-by-djs-nsdc-ultraceuticals/test.csv')
test.head()

,ID,TargetID,DRUGID,DRUGTYPE,Drug_high_status,DRUGNAME,PUBCHCID,Disease_of_highest_status,Drug_Status,UNIPROID,TARGNAME,GENENAME,SYNONYMS,FUNCTION,BIOCLASS,SEQUENCE,Disease,Accession Number
0,1076,T51487,D0HO2T,Small molecular drug,Phase 2,AZD7325,23581869,Anxiety disorder,Phase 2,GBRA1_HUMAN,GABA(A) receptor alpha-1 (GABRA1),GABRA1,Gamma-aminobutyric acid receptor subunit alpha...,Ligand-gated chloride channel which is a compo...,Ligand-gated ion channel,MRKSPGLSDCLWAWILLLSTLTGRSYGQPSLQDELKDNTTVFTRIL...,Malaria,P14867
1,190816,T74312,D0Q4EQ,Small molecular drug,Investigative,JNJ-38158471,16742063,Angiogenesis disorder,Investigative,FLT3_HUMAN,Fms-like tyrosine kinase 3 (FLT-3),FLT3,Stem cell tyrosine kinase 1; STK1; STK-1; Rece...,Tyrosine-protein kinase that acts as cell-surf...,Kinase,MPALARDGGQLPLLVVFSAMIFGTITNQDLPVIKCVLINHKNNDSS...,Malignant haematopoietic neoplasm,P36888
2,180551,T40474,D0R0MW,Small molecular drug,Approved,Sunitinib,5329102,Gastrointestinal stromal tumour,Approved,MET_HUMAN,Proto-oncogene c-Met (MET),MET,Tyrosine-protein kinase Met; Scatter factor re...,Regulates many physiological processes includi...,Kinase,MKAPAVLAPGILVLLFTLVQRSNGECKEALAKSEMNVNMKYQLPNF...,Liver cancer,Q86W50
3,51630,T67162,D0T2XU,Small molecular drug,Approved,Promethazine,4927,Nausea,Approved,DRD2_HUMAN,Dopamine D2 receptor (D2R),DRD2,Dopamine receptor 2; D(2) dopamine receptor,Dopamine receptor whose activity is mediated b...,GPCR rhodopsin,MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...,Depression,P14416
4,50566,T67162,D0H7KF,Small molecular drug,Approved,Quetiapine,5002,Alcohol dependence,Phase 3,DRD2_HUMAN,Dopamine D2 receptor (D2R),DRD2,Dopamine receptor 2; D(2) dopamine receptor,Dopamine receptor whose activity is mediated b...,GPCR rhodopsin,MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...,Lymphoma,P14416


In [10]:
from sklearn.preprocessing import LabelEncoder

for col in le_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    test[col] = test[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1 if pd.notna(x) else -1)
    print("Done for", col)

Done for TargetID
Done for DRUGID
Done for DRUGTYPE
Done for Drug_high_status
Done for DRUGNAME
Done for PUBCHCID
Done for Disease_of_highest_status
Done for Drug_Status
Done for UNIPROID
Done for TARGNAME
Done for GENENAME
Done for SYNONYMS
Done for FUNCTION
Done for BIOCLASS
Done for SEQUENCE
Done for Disease
Done for Accession Number


In [11]:
le = LabelEncoder()
data['Target_Status'] = le.fit_transform(data['Target_Status'])
# test['Target_Status'] = le.transform(test['Target_Status'])

print(le.classes_)

['Application submitted' 'Approved' 'Approved (orphan drug)'
 'BLA submitted' 'Clinical trial' 'Discontinued in Phase 1'
 'Discontinued in Phase 1/2' 'Discontinued in Phase 2'
 'Discontinued in Phase 2/3' 'Discontinued in Phase 2b'
 'Discontinued in Phase 3' 'Discontinued in Phase 4'
 'Discontinued in Preregistration' 'IND submitted' 'Investigative'
 'Patented' 'Phase 0' 'Phase 1' 'Phase 1/2' 'Phase 1/2a' 'Phase 1b'
 'Phase 1b/2a' 'Phase 2' 'Phase 2/3' 'Phase 2a' 'Phase 2b' 'Phase 3'
 'Phase 4' 'Preclinical' 'Preregistration' 'Registered' 'Terminated'
 'Withdrawn from market']


In [13]:
keep_cols = ['ID', 'TargetID', 'SYNONYMS',
             'UNIPROID', 'TARGNAME', 'GENENAME',  
             'FUNCTION', 'BIOCLASS',
             'Disease', 'Accession Number',]

y = data['Target_Status']
data_dropped = data[keep_cols]
# data_dropped.drop('Target_Status', axis=1, inplace=True)
test_dropped = test[keep_cols]

# Without SEQUENCE and Normal Architecture

In [14]:
data_dropped.head()

,ID,TargetID,SYNONYMS,UNIPROID,TARGNAME,GENENAME,FUNCTION,BIOCLASS,Disease,Accession Number
0,140736,332,583,102,676,92,376,101,220,547
1,133048,402,296,516,505,554,145,72,487,256
2,60493,547,564,689,668,371,435,58,479,363
3,169176,636,7,16,24,17,604,45,248,316
4,120183,210,603,26,19,9,557,28,407,249


In [15]:
test_dropped.head()

,ID,TargetID,SYNONYMS,UNIPROID,TARGNAME,GENENAME,FUNCTION,BIOCLASS,Disease,Accession Number
0,1076,339,91,265,245,262,335,61,331,204
1,190816,498,452,254,232,253,667,58,334,366
2,180551,272,538,401,524,412,523,58,318,596
3,51630,451,70,203,189,202,172,45,152,198
4,50566,451,70,203,189,202,172,45,328,198


In [16]:
stacking_clf.fit(data_dropped, y)

0:	learn: 2.2269087	total: 2.48s	remaining: 15m 24s
2:	learn: 5.2934055	total: 6.83s	remaining: 14m 4s
4:	learn: 3.8935985	total: 10.8s	remaining: 13m 19s
6:	learn: 3.1752367	total: 14.9s	remaining: 13m
8:	learn: 3.3435340	total: 18.9s	remaining: 12m 45s
10:	learn: 4.5505761	total: 22.9s	remaining: 12m 35s
12:	learn: 4.1214878	total: 26.9s	remaining: 12m 27s
14:	learn: 3.5958578	total: 30.9s	remaining: 12m 20s
16:	learn: 3.1962251	total: 35.1s	remaining: 12m 17s
18:	learn: 2.0273607	total: 39.3s	remaining: 12m 14s
20:	learn: 1.9943417	total: 43.3s	remaining: 12m 8s
22:	learn: 1.7261496	total: 47.4s	remaining: 12m 2s
24:	learn: 1.3772663	total: 51.4s	remaining: 11m 57s
26:	learn: 1.2707888	total: 55.4s	remaining: 11m 52s
28:	learn: 1.1591744	total: 59.4s	remaining: 11m 46s
30:	learn: 1.1150161	total: 1m 3s	remaining: 11m 41s
32:	learn: 0.9314726	total: 1m 7s	remaining: 11m 38s
34:	learn: 0.8957741	total: 1m 11s	remaining: 11m 35s
36:	learn: 0.8578747	total: 1m 15s	remaining: 11m 30s
38:

StackingClassifier(cv=2,
                   estimators=[('xgb1',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.7824142463846468,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None,
                                              gamma=0.8348828876624594,
                                              grow_policy=None,
                                              importance_type=None,
                                              inte...
                                                 grow_policy=None,
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=None, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 random_state=None, ...))

In [17]:
print('Done')

Done


In [18]:
final_predictions = stacking_clf.predict(test_dropped)

In [19]:
print("Done")

Done


In [20]:
# Define the list of categories in the order you want to encode them
categories = [
    'Application submitted', 'Approved', 'Approved (orphan drug)', 'BLA submitted', 'Clinical trial',
    'Discontinued in Phase 1', 'Discontinued in Phase 1/2', 'Discontinued in Phase 2', 'Discontinued in Phase 2/3',
    'Discontinued in Phase 2b', 'Discontinued in Phase 3', 'Discontinued in Phase 4',
    'Discontinued in Preregistration', 'IND submitted', 'Investigative', 'Patented', 'Phase 0', 'Phase 1', 'Phase 1/2',
    'Phase 1/2a', 'Phase 1b', 'Phase 1b/2a', 'Phase 2', 'Phase 2/3', 'Phase 2a', 'Phase 2b', 'Phase 3', 'Phase 4',
    'Preclinical', 'Preregistration', 'Registered', 'Terminated', 'Withdrawn from market'
]

# Create a dictionary mapping each category to its corresponding label (0, 1, 2, etc.)
category_to_label = {idx : category for idx, category in enumerate(categories)}

# Function to apply the mapping
# def map_to_label(value):
#     return category_to_label.get(value, 1)  # Returns 1 if the value is not found in the dictionary
category_to_label

{0: 'Application submitted',
 1: 'Approved',
 2: 'Approved (orphan drug)',
 3: 'BLA submitted',
 4: 'Clinical trial',
 5: 'Discontinued in Phase 1',
 6: 'Discontinued in Phase 1/2',
 7: 'Discontinued in Phase 2',
 8: 'Discontinued in Phase 2/3',
 9: 'Discontinued in Phase 2b',
 10: 'Discontinued in Phase 3',
 11: 'Discontinued in Phase 4',
 12: 'Discontinued in Preregistration',
 13: 'IND submitted',
 14: 'Investigative',
 15: 'Patented',
 16: 'Phase 0',
 17: 'Phase 1',
 18: 'Phase 1/2',
 19: 'Phase 1/2a',
 20: 'Phase 1b',
 21: 'Phase 1b/2a',
 22: 'Phase 2',
 23: 'Phase 2/3',
 24: 'Phase 2a',
 25: 'Phase 2b',
 26: 'Phase 3',
 27: 'Phase 4',
 28: 'Preclinical',
 29: 'Preregistration',
 30: 'Registered',
 31: 'Terminated',
 32: 'Withdrawn from market'}

In [21]:
sub = pd.DataFrame(test['ID'])
sub['Prediction'] = final_predictions
sub['Prediction'] = sub['Prediction'].map(category_to_label)

sub.to_csv('StackingSubmission1_cv3.csv', index=False)     # 0.99394
sub.head()

,ID,Prediction
0,1076,Approved
1,190816,Phase 3
2,180551,Phase 3
3,51630,Approved
4,50566,Phase 1/2


In [22]:
print("Done!")

Done!


In [24]:
data_dropped.head()

,ID,TargetID,SYNONYMS,UNIPROID,TARGNAME,GENENAME,FUNCTION,BIOCLASS,Disease,Accession Number
0,140736,332,583,102,676,92,376,101,220,547
1,133048,402,296,516,505,554,145,72,487,256
2,60493,547,564,689,668,371,435,58,479,363
3,169176,636,7,16,24,17,604,45,248,316
4,120183,210,603,26,19,9,557,28,407,249


---

# Trying by adding SEQUENCE column and meta is CatBoost

In [26]:
keep_cols2 = ['ID', 'TargetID', 'SYNONYMS',
             'UNIPROID', 'TARGNAME', 'GENENAME',  
             'FUNCTION', 'BIOCLASS', 'SEQUENCE',
             'Disease', 'Accession Number',]

y = data['Target_Status']
data_dropped = data[keep_cols2]
# data_dropped.drop('Target_Status', axis=1, inplace=True)
test_dropped = test[keep_cols2]

In [27]:
data_dropped.head()

,ID,TargetID,SYNONYMS,UNIPROID,TARGNAME,GENENAME,FUNCTION,BIOCLASS,SEQUENCE,Disease,Accession Number
0,140736,332,583,102,676,92,376,101,662,220,547
1,133048,402,296,516,505,554,145,72,600,487,256
2,60493,547,564,689,668,371,435,58,513,479,363
3,169176,636,7,16,24,17,604,45,488,248,316
4,120183,210,603,26,19,9,557,28,537,407,249


In [28]:
test_dropped.head()

,ID,TargetID,SYNONYMS,UNIPROID,TARGNAME,GENENAME,FUNCTION,BIOCLASS,SEQUENCE,Disease,Accession Number
0,1076,339,91,265,245,262,335,61,529,331,204
1,190816,498,452,254,232,253,667,58,459,334,366
2,180551,272,538,401,524,412,523,58,360,318,596
3,51630,451,70,203,189,202,172,45,171,152,198
4,50566,451,70,203,189,202,172,45,171,328,198


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


# Define base models
param1 = {'max_depth': 19, 'learning_rate': 0.018605498363005686, 'n_estimators': 441, 'subsample': 0.9987709321276286,
          'colsample_bytree': 0.7824142463846468, 'gamma': 0.8348828876624594, 'min_child_weight': 3, 'verbosity':2}

param2 =  {'max_depth': 19, 'learning_rate': 0.023064636414923647, 'n_estimators': 447, 'subsample': 0.9992514964718675, 
           'colsample_bytree': 0.7931212254410178, 'gamma': 0.8275527074142363, 'min_child_weight': 3, 'verbosity':2}

param3 = {'iterations': 374, 'depth': 10, 'learning_rate': 0.47497302209777004, 'l2_leaf_reg': 2.8493972804552454, 'verbose':2} 

param4 = {'iterations': 901, 'depth': 9, 'learning_rate': 0.15393581334059922, 'l2_leaf_reg': 0.6611236899302668, 'verbose':2}

param5 = {'iterations': 951, 'depth': 9, 'learning_rate': 0.18509019397073728, 'l2_leaf_reg': 0.12336456448381483, 'verbose':2}

m1 = XGBClassifier(**param1)
m2 = XGBClassifier(**param2)
m3 = CatBoostClassifier(**param3)
m4 = CatBoostClassifier(**param4)
m5 = CatBoostClassifier(**param5)


# Define the meta-learner (can be any classifier)
meta_learner = CatBoostClassifier()

# Create the stacking ensemble
stacking_clf = StackingClassifier(estimators=[
    ('xgb1', m1),
    ('xgb2', m2), 
    ('catboost1', m3),
    ('catboost2', m4),
    ('catboost3', m5),
], final_estimator=meta_learner, cv=2)

# Train the stacking model
stacking_clf.fit(data_dropped, y)

# Make predictions with the stacked model
# final_predictions = stacking_clf.predict(X_test_dropped)

0:	learn: 2.3006559	total: 2.2s	remaining: 13m 39s
2:	learn: 2.6039734	total: 6.49s	remaining: 13m 22s
4:	learn: 2.1137977	total: 10.7s	remaining: 13m 6s
6:	learn: 1.8682260	total: 15.3s	remaining: 13m 24s
8:	learn: 1.7021785	total: 19.5s	remaining: 13m 10s
10:	learn: 1.7811375	total: 23.6s	remaining: 13m
12:	learn: 7.2702131	total: 27.8s	remaining: 12m 52s
14:	learn: 4.2238918	total: 32s	remaining: 12m 46s
16:	learn: 2.9672181	total: 36.2s	remaining: 12m 40s
18:	learn: 2.3588742	total: 40.4s	remaining: 12m 34s
20:	learn: 1.9583530	total: 44.9s	remaining: 12m 33s
22:	learn: 1.7222361	total: 49.1s	remaining: 12m 29s
24:	learn: 1.6010068	total: 53.2s	remaining: 12m 23s
26:	learn: 1.5939251	total: 57.4s	remaining: 12m 17s
28:	learn: 1.4142164	total: 1m 1s	remaining: 12m 12s
30:	learn: 1.1662080	total: 1m 5s	remaining: 12m 7s
32:	learn: 1.1080804	total: 1m 9s	remaining: 12m 2s
34:	learn: 1.0434657	total: 1m 14s	remaining: 11m 57s
36:	learn: 1.0031532	total: 1m 18s	remaining: 11m 56s
38:	le

StackingClassifier(cv=2,
                   estimators=[('xgb1',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.7824142463846468,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None,
                                              gamma=0.8348828876624594,
                                              grow_policy=None,
                                              importance_type=None,
                                              inte...
                                              n_estimators=447, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...)),
                               ('catboost1',
                                <catboost.core.CatBoostClassifier object at 0x7e39fc6e2fe0>),
                               ('catboost2',
                                <catboost.core.CatBoostClassifier object at 0x7e39fc6e2ec0>),
                               ('catboost3',
                                <catboost.core.CatBoostClassifier object at 0x7e39fc6e30d0>)],
                   final_estimator=<catboost.core.CatBoostClassifier object at 0x7e39fc6e31f0>)

In [35]:
print("Done")

Done


In [36]:
final_predictions = stacking_clf.predict(test_dropped)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [37]:
sub = pd.DataFrame(test['ID'])
sub['Prediction'] = final_predictions
sub['Prediction'] = sub['Prediction'].map(category_to_label)

sub.to_csv('StackingSubmission3_cv2.csv', index=False)
sub.head()

,ID,Prediction
0,1076,Approved
1,190816,Phase 3
2,180551,Phase 3
3,51630,Approved
4,50566,Phase 1/2


In [38]:
print("Done")

Done
